### Imports & Definitions

In [1]:
import sys
sys.path.append("../../utils")
from definitions import *
from path_helpers import get_dataset_path, get_metric_dir_path
from mera_helpers import construct_prompt
from huggingface_helpers import get_tokenizer, get_model
from llm_helpers import calculate_token_interest_probs, get_answer

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


### Learn to load datasets from disk

In [2]:
for name, dataset_meta in HUGGINGFACE_NAME_TO_DATASET.items():
    for subset, split in zip(dataset_meta["subsets"], dataset_meta["splits"]):
        path = get_dataset_path(subset, name, split)
        dataset = load_from_disk(path)
        print(subset, dataset[0])

rummlu {'instruction': 'Задание содержит вопрос по теме {subject} и 4 варианта ответа A, B, C, D, из которых только один правильный.\n{text}\nA {option_a}\nB {option_b}\nC {option_c}\nD {option_d}\nЗапишите букву правильного ответа\nОтвет:', 'inputs': {'text': 'В какой из этих двух ситуаций действующее лицо, использующее местоимения "я"/"меня"/"мой", нарушает нормы морали, общепринятые в США по состоянию на 2020 год? Ситуация 1 | Я оторвал голову детской игрушке. Ситуация 2 | Я узнал, что моей сестре нравится один мальчик, она не хотела, чтобы он знал, и я сохранил это в тайне.', 'option_a': 'Нарушает, Нарушает', 'option_b': 'Нарушает, Не нарушает', 'option_c': 'Не нарушает, Нарушает', 'option_d': 'Не нарушает, Не нарушает', 'subject': 'Философия'}, 'outputs': 'B', 'meta': {'domain': 'moral_scenarios', 'id': 0}}
ruopenbookqa {'instruction': '{question}\nA. {option_a}\nB. {option_b}\nC. {option_c}\nD. {option_d}\nКакой ответ является правильным? Запишите только букву верного варианта: A

In [3]:
rummlu = {'instruction': 'Задание содержит вопрос по теме {subject} и 4 варианта ответа A, B, C, D, из которых только один правильный.\n{text}\nA {option_a}\nB {option_b}\nC {option_c}\nD {option_d}\nЗапишите букву правильного ответа\nОтвет:', 'inputs': {'text': 'В какой из этих двух ситуаций действующее лицо, использующее местоимения "я"/"меня"/"мой", нарушает нормы морали, общепринятые в США по состоянию на 2020 год? Ситуация 1 | Я оторвал голову детской игрушке. Ситуация 2 | Я узнал, что моей сестре нравится один мальчик, она не хотела, чтобы он знал, и я сохранил это в тайне.', 'option_a': 'Нарушает, Нарушает', 'option_b': 'Нарушает, Не нарушает', 'option_c': 'Не нарушает, Нарушает', 'option_d': 'Не нарушает, Не нарушает', 'subject': 'Философия'}, 'outputs': 'B', 'meta': {'domain': 'moral_scenarios', 'id': 0}}
construct_prompt(rummlu)

'Задание содержит вопрос по теме Философия и 4 варианта ответа A, B, C, D, из которых только один правильный.\nВ какой из этих двух ситуаций действующее лицо, использующее местоимения "я"/"меня"/"мой", нарушает нормы морали, общепринятые в США по состоянию на 2020 год? Ситуация 1 | Я оторвал голову детской игрушке. Ситуация 2 | Я узнал, что моей сестре нравится один мальчик, она не хотела, чтобы он знал, и я сохранил это в тайне.\nA Нарушает, Нарушает\nB Нарушает, Не нарушает\nC Не нарушает, Нарушает\nD Не нарушает, Не нарушает\nЗапишите букву правильного ответа\nОтвет:'

In [4]:
ruopenbookqa = {'instruction': '{question}\nA. {option_a}\nB. {option_b}\nC. {option_c}\nD. {option_d}\nКакой ответ является правильным? Запишите только букву верного варианта: A, B, C или D.\nОтвет: ', 'inputs': {'question': 'Примером потомства, получившего генетический набор от родителей, является', 'option_a': 'маленький кротенок', 'option_b': 'маленький мешок сахара', 'option_c': 'маленькая тележка', 'option_d': 'маленький пирожок'}, 'outputs': 'A', 'meta': {'id': 0}}
construct_prompt(ruopenbookqa)

'Примером потомства, получившего генетический набор от родителей, является\nA. маленький кротенок\nB. маленький мешок сахара\nC. маленькая тележка\nD. маленький пирожок\nКакой ответ является правильным? Запишите только букву верного варианта: A, B, C или D.\nОтвет: '

### Learn to inference LLM on these datasets

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [27]:
some_russian_models = {
    # SAIGA_MISTRAL_7B_LORA,
    VIKHR_7B,
    # FRED_T5_1_7B,
    # RU_LONGFORMER_LARGE_4096,
    # YALM_100B,
}

In [28]:
import time

costs = dict()
for russian_model in some_russian_models:
    print(russian_model)
    repo = HUGGINGFACE_MODEL_TO_REPO[russian_model]
    tokenizer = get_tokenizer(russian_model, repo)
    model = get_model(russian_model, repo)
    model.to(device)
    q = construct_prompt(rummlu)
    print(q)
    start = time.time()
    probs = calculate_token_interest_probs(q, tokenizer, model)
    finish = time.time()
    costs[russian_model] = finish - start
    print(probs)
    a = get_answer(probs)
    print(a)

vikhr-7b


Loading checkpoint shards: 100%|██████████| 3/3 [00:05<00:00,  1.83s/it]


Задание содержит вопрос по теме Философия и 4 варианта ответа A, B, C, D, из которых только один правильный.
В какой из этих двух ситуаций действующее лицо, использующее местоимения "я"/"меня"/"мой", нарушает нормы морали, общепринятые в США по состоянию на 2020 год? Ситуация 1 | Я оторвал голову детской игрушке. Ситуация 2 | Я узнал, что моей сестре нравится один мальчик, она не хотела, чтобы он знал, и я сохранил это в тайне.
A Нарушает, Нарушает
B Нарушает, Не нарушает
C Не нарушает, Нарушает
D Не нарушает, Не нарушает
Запишите букву правильного ответа
Ответ:
{'A': 0.06380045413970947, 'B': 0.20826202630996704, 'C': 0.10781271010637283, 'D': 0.11802983283996582}
B


In [29]:
for k, v in costs.items():
    print(f"model {k} costs {v:.2f}ms to inference")

model vikhr-7b costs 0.24ms to inference


In [11]:
for k, v in costs.items():
    print(f"model {k} costs {v:.2f}ms to inference")

model ru-e5-base costs 0.02ms to inference
model vikhr-7b costs 0.61ms to inference
model ruElectra-large costs 0.03ms to inference


### Inference LLMs on all datasets

In [30]:
for russian_model in tqdm(some_russian_models, desc="Models..."):
    repo = HUGGINGFACE_MODEL_TO_REPO[russian_model]
    tokenizer = get_tokenizer(russian_model, repo)
    model = get_model(russian_model, repo)
    model.to(device)
    for name, dataset_meta in tqdm(HUGGINGFACE_NAME_TO_DATASET.items(), desc="Datasets..."):
        for subset, split in tqdm(zip(dataset_meta["subsets"], dataset_meta["splits"]), desc="Splits..."):
            path = get_dataset_path(subset, name, split)
            dataset = load_from_disk(path)
            probs_list = []
            a_list = []
            for row in tqdm(dataset, desc="Rows..."):
                q = construct_prompt(row)
                probs = calculate_token_interest_probs(q, tokenizer, model)
                probs_list.append({
                    "probs": probs,
                    "meta": row["meta"],
                })
                a = get_answer(probs)
                a_list.append({
                    "answer": a,
                    "meta": row["meta"],
                })
            metric_dir_path = get_metric_dir_path(russian_model, subset, name, split)
            metric_dir_path.mkdir(exist_ok=True, parents=True)
            with open(metric_dir_path.joinpath("probs.jsonl"), "w") as f:
                json.dump(probs_list, f, ensure_ascii=False, indent=2)
            with open(metric_dir_path.joinpath("answers.jsonl"), "w") as f:
                json.dump(a_list, f, ensure_ascii=False, indent=2)

Models...:   0%|          | 0/1 [00:00<?, ?it/s]

Loading checkpoint shards: 100%|██████████| 3/3 [00:05<00:00,  1.96s/it]

Datasets...:   0%|          | 0/1 [00:00<?, ?it/s]

Splits...: 0it [00:00, ?it/s]


Rows...:   0%|          | 0/10033 [00:00<?, ?it/s]


Rows...:   0%|          | 1/10033 [00:00<41:28,  4.03it/s]


Rows...:   0%|          | 2/10033 [00:00<41:18,  4.05it/s]


Rows...:   0%|          | 3/10033 [00:00<45:04,  3.71it/s]


Rows...:   0%|          | 4/10033 [00:01<46:44,  3.58it/s]


Rows...:   0%|          | 5/10033 [00:01<44:57,  3.72it/s]


Rows...:   0%|          | 6/10033 [00:01<43:41,  3.82it/s]


Rows...:   0%|          | 7/10033 [00:01<41:13,  4.05it/s]


Rows...:   0%|          | 8/10033 [00:01<37:31,  4.45it/s]


Rows...:   0%|          | 9/10033 [00:02<34:58,  4.78it/s]


Rows...:   0%|          | 10/10033 [00:02<36:56,  4.52it/s]


Rows...:   0%|          | 11/10033 [00:02<33:09,  5.04it/s]


Rows...:   0%|          | 12/10033 [00:02<31:56,  5.23it/s]


Rows